# 1. Get the 15 variables from this raster for all Peru departments polygons

In [1]:
pip install rasterio

Note: you may need to restart the kernel to use updated packages.


In [2]:
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from shapely.geometry import mapping

In [3]:
import geopandas as gpd

# Define the relative path to the shapefile from the notebook location
shapefile_path = '../../_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp'

# Read the shapefile using Geopandas
departments = gpd.read_file(shapefile_path)

print(departments.head())

   OBJECTID_1  OBJECTID CCDD   NOMBDEP      CAPITAL  Shape_STAr  Shape_STLe  \
0           1       1.0   01  AMAZONAS  CHACHAPOYAS    3.203006   12.912088   
1           2       2.0   02    ANCASH       HUARAZ    2.954592   11.780424   
2           3       3.0   03  APURIMAC      ABANCAY    1.765933    7.730154   
3           4       4.0   04  AREQUIPA     AREQUIPA    5.330203   17.405040   
4           5       5.0   05  AYACUCHO     AYACUCHO    3.643705   17.127166   

   ORIG_FID  Shape_Leng  Shape_Area                    CORREO  \
0         0   12.912088    3.203006  juan.suyo@geogpsperu.com   
1         1   11.780424    2.954592  juan.suyo@geogpsperu.com   
2         2    7.730154    1.765933  juan.suyo@geogpsperu.com   
3         3   17.405040    5.330203  juan.suyo@geogpsperu.com   
4         4   17.127166    3.643705  juan.suyo@geogpsperu.com   

             CONTACTO   WHATSAPP  \
0  www.geogpsperu.com  931381206   
1  www.geogpsperu.com  931381206   
2  www.geogpsperu.com  931

In [4]:
import geowombat as gw
from rasterio.merge import merge

ModuleNotFoundError: No module named 'geowombat'

In [5]:
# Leer los archivos .tif
raster_files = ['tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif', 'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif','tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif', 'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif', 'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif', 'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11.tif', 'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif', 'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C10.tif', 'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C11.tif', 'tif_files_group4/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R9_C12.tif']

In [ ]:
# Abrir los archivos raster para la unión
rasters_to_merge = [rasterio.open(file) for file in raster_files]

In [ ]:
# Unir los rasters
mosaic, out_transform = merge(rasters_to_merge) #AQUI ES DONDE SE TRABA LA CODIFICACIÓN. HASTA EL CÓDIGO ANTERIOR TODO CORREO BIEN.

In [ ]:
# Mostrar el raster unido
show(mosaic, transform=out_transform)

In [ ]:
# Cerrar los archivos raster
for raster in rasters_to_merge:
    raster.close()

In [ ]:
# Realizar el análisis de zonificación para cada departamento
for index, department in departments.iterrows():
    # Convertir el polígono de departamento a formato que rasterio entiende
    mask = [mapping(department.geometry)]

    # Enmascarar el raster con el polígono del departamento, para obtener solo los valores dentro del departamento
    out_image, out_transform = rasterio.mask.mask(src, mask, crop=True)
    out_meta = src.meta

    # Realización de cálculos
    # ...

In [ ]:
# Cerrar los archivos abiertos
for src in src_files_to_mosaic:
    src.close()